# Data Sourcing

This notebook will make API call and source data.

* S&P 500
   * P/E ratio for S&P (as a proxy for investors' emotional attitude towards the market
* Indian market's version of S&P and Dow Jones
* Market indices based on varying investment philosophies in both countries.


#### Ticker Symbols

* S & P 500 - VOO (Vanguard S&P 500 ETF(Ultra low-cost, strong tracking)
* S & P 500 Growth ETF  - VOOG (Exposure to Large-Cap US Growth companies)
* JP Morgan Equity Premium Income ETF (Actively managed) - JEPI


In [1]:
import pandas as pd, requests

In [2]:
apiKey = 'RQRA0RQ6ZZAVMA26'

In [3]:
tickers = ['VOO','VOOG','JEPI']

In [50]:
def fetchAPI(tickers, apiKey):
    
    r = requests.get(f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={'VOO'}&outputsize=full&apikey={apiKey}")

    # Contacts API endpoint with the API key and tickers

    apiData = r.json()

    # Converts the JSON data returned by the API to a Python readable format
    
    keys = list(apiData.keys())

    # extracts the keys from the dict returned by the API call
    
    return apiData[keys[1]]

    # returns the value from the second key value pair in the dict containing the data we are interested in.

In [51]:
fetchAPI('VOO',apiKey)

IndexError: list index out of range

In [52]:
# def clean_api_data(data, tickers):
data = fetchAPI('VOO',apiKey)
df = pd.DataFrame.from_dict(data, orient = 'index').reset_index()
# df['index'] = pd.to_datetime(df['date'])

IndexError: list index out of range

callAPI function:

Will take in a list of ticker symbols and return a dataframe.


In [5]:
def callAPI(tickers): 
    
    dfs =[]
    
    for ticker in tickers:
        # API request
        r = requests.get(
            f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={ticker}&outputsize=full&apikey={apiKey}"
        )
        APIdata = r.json()

        # The time series is usually the 2nd key in the dict
        keys = list(APIdata.keys())
        data = APIdata[keys[1]]

        # Convert to DataFrame
        temp_df = pd.DataFrame.from_dict(data, orient='index')

        # Reset index so 'date' becomes a column
        temp_df.reset_index(inplace=True)
        temp_df.rename(columns={'index': 'date'}, inplace=True)

        # Rename columns to cleaner names
        rename_map = {
            '1. open': 'open',
            '2. high': 'high',
            '3. low': 'low',
            '4. close': 'close',
            '5. volume': 'volume'
        }
        temp_df.rename(columns=rename_map, inplace=True)

        # Convert types
        temp_df['date'] = pd.to_datetime(temp_df['date'])
        temp_df[['open', 'high', 'low', 'close']] = temp_df[['open', 'high', 'low', 'close']].astype(float)
        temp_df['volume'] = temp_df['volume'].astype(int)

        # Add ticker column
        temp_df['ticker'] = ticker

        dfs.append(temp_df)

    # Combine all tickers into one DataFrame
    all_data = pd.concat(dfs, ignore_index=True)

    return all_data

In [6]:
callAPI(tickers)

,date,open,high,low,close,volume,ticker
0,2025-08-26,590.38,593.4300,589.795,593.060,4776577,VOO
1,2025-08-25,592.05,593.2299,590.550,590.700,6483035,VOO
2,2025-08-22,586.27,594.3200,585.840,593.210,6389278,VOO
3,2025-08-21,584.93,586.4600,582.650,584.270,4807277,VOO
4,2025-08-20,587.81,588.0000,581.850,586.580,7113572,VOO
...,...,...,...,...,...,...,...
8846,2020-05-28,51.06,56.2700,50.920,51.140,53336,JEPI
8847,2020-05-27,50.72,50.7200,50.400,50.627,352088,JEPI
8848,2020-05-26,50.50,50.6000,50.238,50.238,5638,JEPI
8849,2020-05-22,49.93,50.1000,49.890,50.094,9518,JEPI
